<a href="https://colab.research.google.com/github/SwarajShinde/Buyers-s-Time-Prediction-Challenge/blob/main/statistical_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sklearn 
import pandas as pd
import numpy as np
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Regress/reg_problems/ParticipantData_BTPC/Train.csv")
test = pd.read_csv("/content/drive/MyDrive/Regress/reg_problems/ParticipantData_BTPC/Test.csv")
sub = pd.read_csv("/content/drive/MyDrive/Regress/reg_problems/ParticipantData_BTPC/Sample Submission.csv")

In [ ]:
test

,session_id,session_number,client_agent,device_details,date,purchased,added_in_cart,checked_out
0,0bf72b05b2cf70dc5e28c3c09f9c4f97,99,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,Desktop - IE,2020-01-07,1,0,0
1,6bc696d99954e4f1367e845465186aef,935,Product/160393 Android/4.3 Device/samsung_GT-I...,Android Phone - Android,2020-02-13,0,0,0
2,3d8611a5ea0f621d205b67f09f663a89,902,Product/4.2.2 iPhone/7.1.2,iPhone - iOS,2019-10-14,1,0,0
3,a0351545c032fd0da36289410c6c3354,3223,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Desktop - IE,2020-01-26,0,0,0
4,dd8999772b0ab27225fb05e5df2f1da6,231,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,Desktop - Chrome,2020-01-29,0,0,0
...,...,...,...,...,...,...,...,...
2322,a7ce384215acf0a0efe8e6152d6e0af3,1375,Product/244 CFNetwork/711.1.12 Darwin/14.0.0,Other - Other,2019-11-06,0,0,0
2323,627898fc1c329846956412487ca617b3,1617,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,Desktop - Chrome,2020-01-18,0,1,0
2324,f642380795f4b57ffac8b5c4d6556765,4433,Product/4.7.0 iPhone/8.1.2,iPhone - iOS,2020-01-14,0,0,0
2325,298c00625e7ffef797745a866b898d3c,297,Product/4.2.2 iPad/8.0,iPad - iOS,2019-10-01,1,0,0


In [ ]:
print(train['client_agent'].isna().sum())
print(train['session_id'].isna().sum())
print(train['session_number'].isna().sum())
print(train['device_details'].isna().sum())
print(train['date'].isna().sum())


160
0
0
0
0


In [ ]:
train.shape

(5429, 9)

In [ ]:
nan_train = train[train['client_agent'].isna()]

In [ ]:
train['device_details'].nunique()

17

## **Lets Generate a S1mple Baseline**
1. Exclude the client_agent feature

In [ ]:
train_df = train.drop('client_agent',axis=1)

In [ ]:
train_df = train.drop('session_number',axis=1)

In [ ]:
train_df = train_df.drop('client_agent',axis=1)

In [ ]:
train_df.head(n=7)

,session_id,device_details,date,purchased,added_in_cart,checked_out,time_spent
0,57f879e70d3c5fc2a98102d64c9fd84e,Desktop - Chrome,2020-01-22,1,0,0,236.886
1,a5442b0c7c33d0a811e7661e556b2de8,iPhone - iOS,2020-02-27,1,0,0,1277.455
2,305cb1486ed8610c00b37007926cb2c4,iPhone - MobileWeb,2019-08-01,0,0,0,21.704
3,f2c1ecc9993f0071df91ba178450498c,Desktop - IE,2019-12-30,0,1,0,455.201
4,e460830ae295e55d2216ebdc761ab9a6,iPhone - Web,2019-09-10,0,0,0,1164.877
5,b4ae55a4553d43697dcf64b66033677b,Android Phone - Android,2020-02-06,0,1,0,6010.343
6,a17491477ea11e951286f583b3095899,Desktop - Firefox,2019-05-30,1,1,1,8471.435


In [ ]:
from sklearn.preprocessing import LabelEncoder
lbl=LabelEncoder()

In [ ]:
train_df['device_details'] = lbl.fit_transform(train_df['device_details'])

In [ ]:
train_df.head()

,session_id,device_details,date,purchased,added_in_cart,checked_out,time_spent
0,57f879e70d3c5fc2a98102d64c9fd84e,6,2020-01-22,1,0,0,236.886
1,a5442b0c7c33d0a811e7661e556b2de8,16,2020-02-27,1,0,0,1277.455
2,305cb1486ed8610c00b37007926cb2c4,14,2019-08-01,0,0,0,21.704
3,f2c1ecc9993f0071df91ba178450498c,8,2019-12-30,0,1,0,455.201
4,e460830ae295e55d2216ebdc761ab9a6,15,2019-09-10,0,0,0,1164.877


In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
def metric(true_vals,preds):
    return np.sqrt(mean_squared_log_error(true_vals,preds))

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model=LinearRegression()

In [ ]:
# Create Cross Validation Score:
from sklearn.model_selection import train_test_split
train_,val_ = train_test_split(train_df,test_size=0.1,random_state=25)

In [ ]:
print(train_.shape,val_.shape)

(4886, 7) (543, 7)


In [ ]:
feats=train_[['device_details','purchased','added_in_cart','checked_out']]
vals=train_['time_spent']

In [ ]:
model.fit(X=feats,y=vals)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
val_features=val_[['device_details','purchased','added_in_cart','checked_out']]
val_tars = val_['time_spent']

In [ ]:
val_preds=model.predict(X=val_features)

In [ ]:
metric(val_tars,val_preds)

2.078346113743301

#Lets Try out new Model Architechtures

In [ ]:
from sklearn import linear_model
model1 = linear_model.Lasso(alpha=0.1)

In [ ]:
model1.fit(X=feats,y=vals)
val_preds1=model1.predict(X=val_features)

In [ ]:
metric(val_tars,val_preds1)

2.078636560313991

In [ ]:
from sklearn.svm import SVR
model2=SVR(C=1.0,epsilon=0.2)


In [ ]:
model2.fit(X=feats,y=vals)
val_preds2 = model2.predict(X=val_features)


In [ ]:
print(val_preds2[0],val_preds1[0],val_preds[0])

153.47758325029844 302.156290596448 301.8460769252589


In [ ]:
metric(val_tars,val_preds2)
# Actual LB Score : 1.85203

1.7766825690199008

# Lets keep the Model the basic one (linear reg) :
1. Use the features that were skipped at first 
2. also we can use date if poosible

#Test and Submission Part

In [ ]:
test['device_details'] = lbl.transform(test['device_details'])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:576: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


ValueError: ignored

In [ ]:
dev_det = train_df['device_details'].unique()
test_det=test['device_details'].unique()

In [ ]:
print(sorted(dev_det))
print(sorted(test_det))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [ ]:
test_feats=test[['device_details','purchased','added_in_cart','checked_out']]


In [ ]:
y_preds=model4.predict(X=test_feats)

In [ ]:
y_preds.shape

(2327,)

In [ ]:
test.shape

(2327, 8)

In [ ]:
sub['time_spent'] = y_preds

In [ ]:
sub.head(n=2)

,time_spent
0,473.770329
1,121.932742


In [ ]:
sub.to_csv('today2.csv', index=False)


# Lets FIND Good Hyperparamets using Grid/Random search CV
1. Also use Cross-Validation

In [ ]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']} 

In [ ]:
grid=GridSearchCV(model2,param_grid,refit=True,verbose=3)
grid.fit(X=feats,y=vals)

In [ ]:
grid.best_params_

{'C': 1000, 'gamma': 1, 'kernel': 'rbf'}

In [ ]:
model4 = SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma=1,kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
model4.fit(X=feats,y=vals)
val_preds4 = model4.predict(X=val_features)


In [ ]:
print(metric(val_tars,val_preds4))


1.4721162984555771


# ROUGH Enseble based on sub2,sub4

In [ ]:
from sklearn.svm import SVR
model_svm_1=SVR(C=1000,epsilon=0.2,kernel='poly')


In [ ]:
model_svm_1.fit(X=feats,y=vals)
svm_preds = model_svm_1.predict(X=val_features)


In [ ]:
metric(val_tars,svm_preds)

1.6087560310407845

In [ ]:
svm_test_preds2 = model_svm_1.predict(X=test_feats)

In [ ]:
svm_test_preds2[10:15]

array([128.90243534, 107.82956949,  53.65640857, 112.45828893,
       112.45828893])

In [ ]:
pred1=pd.read_csv("/content/my_submission_file2.csv")
pred2 = pd.read_csv("/content/today.csv")

In [ ]:
pred = pred1['time_spent']+pred2['time_spent']

In [ ]:
sub['time_spent'] = pred/2

In [ ]:
sub.to_csv('min_ens_22.csv',index=False)

In [ ]:
ress = pd.read_csv("/content/sample_submission.csv")

In [ ]:
val_preds=val_preds2+val_preds4+svm_preds

In [ ]:
val_preds[1]

142.57824571782248

In [ ]:
val_preds2[1]+val_preds4[1]+svm_preds[1]

142.57824571782248

In [ ]:
val_preds=val_preds/3
val_preds[1]

47.52608190594083

In [ ]:
metric(val_tars,val_preds)

1.5066609750960704

In [ ]:
# cross val metric seems to be having 1.628 value

In [ ]:
op1 = pd.read_csv('/content/today1.csv')
op2 = pd.read_csv('/content/today2.csv')
op3 = pd.read_csv('/content/today3.csv')

In [ ]:
pred = op1['time_spent']+op2['time_spent']+op3['time_spent']


In [ ]:
sub['time_spent'] = pred/3
sub.to_csv('simple_sub1.csv',index=False)

lets check ensemble strategy

In [ ]:
val_preds=val_preds2+val_preds4
val_preds=val_preds/2

In [ ]:
metric(val_tars,val_preds)

1.515716866178937